In [1]:
from optimize_model import optimize_model
from cobra_utils import print_logo, load_model, set_default_bounds, \
    convert_model_format, fetch_norm_sample_metabolomics_data, match_names_to_vmh

# Define paths
json_model_path = "example_data/models/microbiota_model_diet_Case_1_18_month.json"
matlab_model_path = "example_data/models/microbiota_model_diet_Case_1_18_month.mat"
metabolomics_path = "example_data/metabolomics_data.csv"
avg_eu_diet_path = "example_data/AverageEU_diet_fluxes.txt"

# Load the model
model = load_model(json_model_path)


[START] Loading model from example_data/models/microbiota_model_diet_Case_1_18_month.json...
Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-08

[DONE] microbiota_model_diet_Case_1_18_month loaded.


In [3]:
fetch_norm_sample_metabolomics_data(model_input=model, gcms_filepath=metabolomics_path, match_key_output_filepath="example_outputs")


[START] Fetching metabolomics data for microbiota_model_diet_Case_1_18_month...

[START] Matching GC-MS names to VMH identifiers...

	[1/3] Direct matching of GC-MS names to VMH identifiers using the VMH database.

	[2/3] Matching of GC-MS names to VMH identifiers via PubChemPy.
	Matched 3-Hydroxyphenylacetic acid to 3hphac using InChIKey
	Matched N-Acetyl-D-galactosamine to acgal using InChIKey
	Matched Arabinose (peak 1) to arab_L using InChIKey
	Matched Arabinose (peak 2) to arab_L using InChIKey
	Matched Arabinopyranose to arab_L using InChIKey
	Matched Glucose to glc_D using InChIKey
	Matched Glycine to gly using InChIKey
	Matched Glycerol to glyc using InChIKey
	Matched Pentanoic acid to M03134 using InChIKey
	Matched Mannose to man using InChIKey
	Matched Proline to pro_L using InChIKey
	Matched Serine to ser_L using InChIKey
	Matched Threonine to thr_L using InChIKey
	Matched 3-Hydroxyphenylacetic acid to 3hphac using CID
	Matched alpha-Ketoisovaleric acid to 3mob using CID
	M

{'2hiv': 0.013183485830284992,
 '3mop': 0.04183084707675532,
 '2hxic_L': 0.010839315607069269,
 'pppn': 0.00738734314695842,
 'caproic': 0.009383192418280444,
 'mal_L': 0.010374018999535377,
 'lac_D': 0.01893234795526687,
 'drib': 0.0405712708056671,
 'arabt_L': 0.007732671747365737,
 'rib_D': 0.05241219529336125,
 'phlac': 0.009951907783875375,
 'fru': 0.051727361236636314,
 'srb_L': 0.006802921448209613,
 'tagat_D': 0.003922291441618361,
 'all_D': 0.01737775813939799,
 'galct_D': 0.012036375614709598,
 'gal': 0.01568435412147774,
 '3hphac': 0.005200545324449046,
 'acgal': 0.006193092859023613,
 'arab_L': 0.008099059196965951,
 'glc_D': 0.017821370735104508,
 'gly': 0.012057013005946665,
 'glyc': 0.034616793392635035,
 'M03134': 0.02072290505630485,
 'man': 0.00520420498436549,
 'pro_L': 0.01648771620398017,
 'ser_L': 0.015585838324302834,
 'thr_L': 0.026084622927361963,
 '3mob': 0.0035099159047379982,
 'asp_L': 0.013374441405771605,
 'but': 0.006148362121311685,
 'bz': 0.004057052622

In [ ]:
#########################################################
# Part 3: maximize the flux of fecal exchange reactions
#########################################################

print(f"\n[STARTED] Part 3: maximizing EX_met[fe] fluxes for {model.name}")

# Fetch all UFEt reactions and store them in a list
FEX_rxn_list = []
for rxn in model.reactions:
    if rxn.id.startswith("EX_") and rxn.id.endswith("[fe]"):
        FEX_rxn_list.append(rxn.id)
        print(f"Added {rxn.id} to the list of FEX reactions.")

In [ ]:
# Maximize the flux through all UFEt reactions
counter = 0
counter_max = len(FEX_rxn_list)
maximized_FEX_flux_list = []
for rxn in FEX_rxn_list:
    counter += 1
    print(
        f"\nMaximizing fecal exchange reaction {str(counter)} of {str(counter_max)} for {model.name}"
    )
    model.objective = rxn
    solution = model.optimize()
    maximized_FEX_flux_list.append(solution.objective_value)
    print(f"{rxn}:\t{solution.objective_value}")

# Create a dictionary of the maximized UFEt fluxes
maximized_UFEt_flux_dict = dict(zip(FEX_rxn_list, maximized_FEX_flux_list))

print(f"\n[COMPLETED] Part 1: maximization complete for {model.name}")